# Obtendo dados do site MaisTocadas anualmente

Neste notebook, vamos buscar as listas de músicas mais tocadas nas rádios anualmente de 1980 a 2016.

In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from datetime import timedelta, date

In [ ]:
url = 'https://maistocadas.mus.br/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
}

In [ ]:
musicas_populares = []

for ano in range(1980, 2017):
    r = requests.get(url + str(ano), {'headers': headers})
    soup = bs4.BeautifulSoup(r.content, 'html.parser')

    posicao = 1
    for li in soup.find('ol'):
        faixa = li.find('span', {'class': 'musicas'}).get_text(strip=True)
        artista = li.find('span', {'class': 'artista'}).get_text(strip=True)
        
        musicas_populares.append({
            'posicao': posicao,
            'musica': faixa,
            'artista': artista,
            'ano': ano
        })
        posicao += 1


In [ ]:
df = pd.DataFrame(musicas_populares, columns=['posicao', 'musica', 'artista', 'ano'])
df.head(5)

In [ ]:
df.to_excel('musicas-1980-2016.xlsx', engine='openpyxl')

# Buscando listas do site Spotify Charts

Agora, vamos buscar as músicas mais ouvidas no serviço de streaming Spotify entre os anos de 2017 e 2021. Através do site SpotifyCharts é possível obter semanalmente a lista de músicas mais ouvidas e no final fazer um somatório ano a ano.


In [ ]:
nome_arquivo = 'content/regional-br-weekly-{0}--{1}.csv' #regional-br-weekly-2017-01-13--2017-01-20
url = 'https://spotifycharts.com/regional/br/weekly/{0}--{1}/download'
data_final = date(2021,1,1)
data = date(2016,12,30)

dataframe_2017 = pd.DataFrame(columns=['Position',	'Track Name',	'Artist',	'Streams', 'URL', 'Week'])
dataframe_2018 = pd.DataFrame(columns=['Position',	'Track Name',	'Artist',	'Streams', 'URL', 'Week'])
dataframe_2019 = pd.DataFrame(columns=['Position',	'Track Name',	'Artist',	'Streams', 'URL', 'Week'])
dataframe_2020 = pd.DataFrame(columns=['Position',	'Track Name',	'Artist',	'Streams', 'URL', 'Week'])

while (data < data_final):
  inicio = data 
  fim = data + timedelta(7)

  df = pd.read_csv(nome_arquivo.format(inicio.strftime("%Y-%m-%d"), fim.strftime("%Y-%m-%d")), skiprows=[0])
  df['Week'] = inicio
  if (inicio.strftime("%Y") == '2017'):
    dataframe_2017 = pd.concat([dataframe_2017, df])
  elif (inicio.strftime("%Y") == '2018'):
    dataframe_2018 = pd.concat([dataframe_2018, df])
  elif (inicio.strftime("%Y") == '2019'):
    dataframe_2019 = pd.concat([dataframe_2019, df])
  elif (inicio.strftime("%Y") == '2020'):
    dataframe_2020 = pd.concat([dataframe_2020, df])

  data = fim

In [ ]:
dataframe_2017 = dataframe_2017.groupby(by=['Track Name', 'Artist'], sort='Streams')['Streams'].sum()
dataframe_2017 = pd.DataFrame(dataframe_2017).sort_values(by='Streams', ascending=False)

dataframe_2018 = dataframe_2018.groupby(by=['Track Name', 'Artist'], sort='Streams')['Streams'].sum()
dataframe_2018 = pd.DataFrame(dataframe_2018).sort_values(by='Streams', ascending=False)

dataframe_2019 = dataframe_2019.groupby(by=['Track Name', 'Artist'], sort='Streams')['Streams'].sum()
dataframe_2019 = pd.DataFrame(dataframe_2019).sort_values(by='Streams', ascending=False)

dataframe_2020 = dataframe_2020.groupby(by=['Track Name', 'Artist'], sort='Streams')['Streams'].sum()
dataframe_2020 = pd.DataFrame(dataframe_2020).sort_values(by='Streams', ascending=False)

In [ ]:
dataframe_2017.to_excel('spotify-2017.xlsx', engine='openpyxl')
dataframe_2018.to_excel('spotify-2018.xlsx', engine='openpyxl')
dataframe_2019.to_excel('spotify-2019.xlsx', engine='openpyxl')
dataframe_2020.to_excel('spotify-2020.xlsx', engine='openpyxl')